# Clojure

[Clojure](http://clojure.org/) is a dialect of Lisp that runs in the JVM.
It shares with Lisp the code-as-data philosophy and a powerful macro system.
Clojure is predominantly a functional programming language, and features a rich set of immutable, persistent data structures.
It has strong support for reliable multithreading and concurrency.

In [1]:
; lazy infinite sequence with recursive definition
(def fib-seq-lazy 
  ((fn rfib [a b] 
     (lazy-seq (cons a (rfib b (+ a b)))))
   0 1))
(take 20 fib-seq-lazy)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

In [2]:
; 'unless' cannot be defined with a function because
; it does not always evaluate both its arguments.
(defmacro unless [pred a b]
  `(if (not ~pred) ~a ~b))
(unless false (println "Will print") (println "Will not print"))

Will print


null

In [3]:
(defn run [nvecs nitems nthreads niters]
  (let [vec-refs (vec (map (comp ref vec)
                           (partition nitems (range (* nvecs nitems)))))
        swap #(let [v1 (rand-int nvecs)
                    v2 (rand-int nvecs)
                    i1 (rand-int nitems)
                    i2 (rand-int nitems)]
                (dosync
                 (let [temp (nth @(vec-refs v1) i1)]
                   (alter (vec-refs v1) assoc i1 (nth @(vec-refs v2) i2))
                   (alter (vec-refs v2) assoc i2 temp))))
        report #(do
                 (prn (map deref vec-refs))
                 (println "Distinct:"
                          (count (distinct (apply concat (map deref vec-refs))))))]
    (report)
    (dorun (apply pcalls (repeat nthreads #(dotimes [_ niters] (swap)))))
    (report)))
; When run, we see no values get lost or duplicated in the shuffle.
; There are 36 distinct numbers before and after. 
(run 6 6 6 100000)

([0 1 2 3 4 5] [6 7 8 9 10 11] [12 13 14 15 16 17] [18 19 20 21 22 23] [24 25 26 27 28 29] [30 31 32 33 34 35])
Distinct: 36
([31 0 24 13 17 20] [11 4 22 25 12 34] [21 29 28 30 14 18] [16 33 10 26 32 15] [3 8 1 6 7 35] [19 5 27 9 2 23])
Distinct: 36


null

## Using the classpath to load a jar

This first cell shows that you get an error if you try to import a class not built-in to BeakerX:

In [4]:
(import com.beaker.BeakerXClasspathTest)
(def classpathTest (new BeakerXClasspathTest))

java.lang.ClassNotFoundException:  com.beaker.BeakerXClasspathTest

The magic `%classpath` allows you to add jars to your kernel.

In [5]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [6]:
(import com.beaker.BeakerXClasspathTest)
(def classpathTest (new BeakerXClasspathTest))

#'beaker_clojure_shell_d26ca152-c349-4e19-9491-8a803ca3c6c8/classpathTest

In [7]:
(BeakerXClasspathTest/staticTest)

static_123

In [8]:
(. classpathTest getObjectTest)

object_123

## Array of Maps with Properties

In [9]:
[{:foo 1}{:foo 2}]

## Interactive Plot

In [10]:
(import '[com.twosigma.beakerx.chart.xychart Plot]
        '[com.twosigma.beakerx.chart.xychart.plotitem Line])
(doto (Plot.)
            (.setTitle "We Will Control the Title")
            (.setXLabel "Horizontal")
            (.setYLabel "Vertical")
            (.add (doto (Line.)
                        (.setX [0, 1, 2, 3, 4, 5])
                        (.setY [0, 1, 6, 5, 2, 8]))))

## EasyForm

In [11]:
(def form (doto (com.twosigma.beakerx.easyform.EasyForm. "Test Form")
                (.addTextField "Name")
                (.addButton "Reverse" "reverse")))
form

In [12]:
(reverse (get form "Name"))

[]

## Import/Unimport magic command

In [32]:
%import com.twosigma.beakerx.widget.IntSlider

In [33]:
(new IntSlider)

In [34]:
%unimport com.twosigma.beakerx.widget.IntSlider

java.lang.RuntimeException:  Could not create class while loading notebook

In [35]:
(new IntSlider)

java.lang.IllegalArgumentException:  Unable to resolve classname

## IPyVolume visualization widget

[ipyvolume](https://github.com/maartenbreddels/ipyvolume) works with Clojure, and the other [3D widgets](../groovy/3D.ipynb) probably work too.  Note, the widget JS needs to be installed in advance of starting the notebook:
```
conda install -c conda-forge ipyvolume
```
See the [source of this example](https://github.com/maartenbreddels/ipyvolume/pull/140#issuecomment-394777662).

In [36]:
%classpath add mvn com.github.twosigma ipyvolume master-SNAPSHOT

Could not resolve dependencies for:
com.github.twosigma : ipyvolume : master-SNAPSHOT : jar


In [37]:
(import ipyvolume.PyLab)

(defn ball 
    [size radius]
    (let [data           (make-array Float/TYPE size size size)
          square         #(* % %)
          center         #(- % (/ size 2))
          sum            (partial reduce +)
          sum-of-squares #(sum (map square %))]
        (doseq [i (range size)
                j (range size)
                k (range size)
                :let [[x y z] (map center [i j k])] 
                :when (> (square radius) 
                         (sum-of-squares [x y z]))]
             (aset data i j k (float 1)))
        data))
(PyLab/volShow (ball 32 12))

java.lang.ClassNotFoundException:  ipyvolume.PyLab